# On Jean Zay HPC

```bash
module purge

module load arch/h100
module load cuda/12.4.1
module load ffmpeg/6.1.1

module load miniforge

conda activate jsalt10
```

If you want to perform dScaper and PyRoomAcoustics (steps 2 and 3):
```bash
cd ./dscaper
pip install -e .

conda install sox
pip install sox

pip install jams
pip install pyloudnorm
```

# SDialog dependencies

In [ ]:
%%script false --no-raise-error
# Only run this cell if you are using a non jupyter environment
!conda create --name jsalt python=3.9 -y
!conda activate jsalt
!conda install sox
!pip install -r sdialog/requirements.txt
!pip install -r sdialog/requirements-audio.txt
#  conda activate jsalt-Py3-10

In [ ]:
import os
import json
from tqdm import tqdm

In [ ]:
import sdialog
from sdialog import Dialog
from sdialog.generators import PersonaGenerator
from sdialog.personas import Persona, PersonaAgent, Doctor, Patient, Agent

# Generate Persona

In [ ]:
sdialog.config.set_llm("aws:anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-east-1")

In [ ]:
%%script false --no-raise-error
persona_card_folder = "./personas"

# Generate doctor persona
doctor_persona = Doctor(
    name="Dr. Smith",
    gender="male",
    age=52,
    specialty="Family Medicine"
)
generator_doctor = PersonaGenerator(doctor_persona)
persona_cards = generator_doctor.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_doctor.json")

# Generate patient persona
patient_persona = Patient(
    name="John Doe",
    gender="male",
    age=62
)
generator_patient = PersonaGenerator(patient_persona)
persona_cards = generator_patient.generate(n=1)
persona_cards.to_file(f"{persona_card_folder}/persona_patient.json")


# Load persona

In [ ]:
# Load personas
persona_doctor = Persona.from_file("./personas/persona_doctor.json")
persona_patient = Persona.from_file("./personas/persona_patient.json")

In [ ]:
%%script false --no-raise-error
context = "Generate me a 50 turn medical dialogue between patient and doctor, for a primary care visit"

# Create agents
agent1 = PersonaAgent(persona=persona_doctor, name="DOCTOR", dialogue_details=context, response_details="make short turn answers when needed")
agent2 = PersonaAgent(persona=persona_patient, name="PATIENT", dialogue_details=context, response_details="make short turn answers when needed")

In [ ]:
save_all = True
GENERATE_PERSONA = True
FORCE_DIALOG_GENERATION = False

os.makedirs("./outputs", exist_ok=True)

In [ ]:
if FORCE_DIALOG_GENERATION:

    original_dialog = agent1.talk_with(agent2, max_turns=3)
    original_dialog.to_file("dialog_demo.json")

else:
    original_dialog = Dialog.from_file("dialog_demo.json")

original_dialog.print()

# Audio Generation

You can generate three type of audios:
- (default) Step 1: Raw utterances passed to a TTS model and concatenated to each others to create an audio file
- Step 2: Audio generated from multiple channels create using signal positions
- Step 3: Audio generated using room spacialization and multi-channels positions

If you want to trigger the "step 2" you need to give a Scaper argument to the `audio_pipeline`. While for the "step 3" you need also to give a "Room" in the `inference` function.

### Instanciate voices database

In [ ]:
from sdialog.audio.voice_database import DummyKokoroVoiceDatabase
dummy_voice_database = DummyKokoroVoiceDatabase()
dummy_voice_database.get_voice(genre="male", age=20)

OR

In [ ]:
%%script false --no-raise-error
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
voices_libritts = HuggingfaceVoiceDatabase("sdialog/voices-libritts")
voices_libritts.get_voice(genre="male", age=20)

OR

In [ ]:
%%script false --no-raise-error
from sdialog.audio.voice_database import HuggingfaceVoiceDatabase
dummy_voice_database = HuggingfaceVoiceDatabase("sdialog/voices-jsalt")
dummy_voice_database.get_voice(genre="male", age=20)

### Instanciate TTS model

In [ ]:
from sdialog.audio.tts_engine import KokoroTTS
tts_pipeline = KokoroTTS()

In [ ]:
%%script false --no-raise-error
##################################################
# DOESN'T WORK ON MULTILINGUAL MACOS
##################################################

# Generate multilingual audio from text using the Kokoro model

from sdialog.audio.tts_engine import KokoroTTS

tts_pipeline = KokoroTTS(lang_code="f")
# tts_pipeline = KokoroTTS(lang_code="a")

from phonemizer.backend.espeak.wrapper import EspeakWrapper
_ESPEAK_LIBRARY = '/opt/homebrew/Cellar/espeak/1.48.04_1/lib/libespeak.1.1.48.dylib'
EspeakWrapper.set_library(_ESPEAK_LIBRARY)

import soundfile
audio, sampling_rate = tts_pipeline.generate(
    # "Hi, how are you today?",
    # "af_alloy"
    "Bonjour, comment ça va?",
    "ff_siwis"
)
print(audio)
print(sampling_rate)
output_file_name = "./test_index_tts_french.wav"
soundfile.write(output_file_name, audio, sampling_rate)
print(f"Audio saved to {output_file_name}")

OR

In [ ]:
%%script false --no-raise-error
from sdialog.audio.tts_engine import IndexTTS
tts_pipeline = IndexTTS(device="cpu")

In [ ]:
%%script false --no-raise-error
# Generate audio from text using the IndexTTS model
import soundfile
audio, sampling_rate = tts_pipeline.generate(
    "Brno is the best city in the planet, you know? and Loco Polaco is the craziest person I know",
    "./sergio.wav"
)
soundfile.write("./test_index_tts.wav", audio, sampling_rate)

## Setup stage: Audio Dialog and Audio Pipeline

In [ ]:
from sdialog.audio.audio_dialog import AudioDialog
from sdialog.audio.audio_pipeline import AudioPipeline

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)

## Step 1 : Concatenated utterances

In [ ]:
audio_pipeline = AudioPipeline(
    voice_database=dummy_voice_database,
    tts_pipeline=tts_pipeline,
    dir_audio="./outputs",
)

OR

In [ ]:
%%script false --no-raise-error
audio_pipeline = AudioPipeline() # Default values are used

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(dialog) # Generate the audio for the dialog
print(dialog.audio_step_1_filepath) # Path to the audio of the first stage of the audio pipeline

## Step 2: dScaper

In [ ]:
%%script false --no-raise-error
!git clone https://github.com/cyrta/dscaper.git

In [ ]:
%%script false --no-raise-error
%pip install -r ../../../requirements-dscaper.txt

In [ ]:
import scaper
DATA_PATH = "./dscaper_data" # Path where the sound events, utterances and timelines database will be saved
os.makedirs(DATA_PATH, exist_ok=True)

In [ ]:
dsc = scaper.Dscaper(dscaper_base_path=DATA_PATH)

In [ ]:
audio_pipeline = AudioPipeline(dscaper=dsc)

In [ ]:
# Populate the sound events database
audio_pipeline.populate_dscaper(["sdialog/background","sdialog/foreground"])

In [ ]:
dialog: AudioDialog = audio_pipeline.inference(dialog)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)

## Step 3 : Room Accoustics

In [ ]:
audio_pipeline = AudioPipeline(dscaper=dsc) # The audio pipeline doesn't change

In [ ]:
from sdialog.audio.room import MicrophonePosition
from sdialog.audio.room_generator import RoomGenerator, RoomRole

In [ ]:
room = RoomGenerator().generate(RoomRole.CONSULTATION, room_size=8.0)
print(room)
print("--------------------------------")
print(room.name)

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    room=room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
    # microphone_position=MicrophonePosition.MONITOR # Default is MicrophonePosition.MONITOR
    microphone_position=MicrophonePosition.CEILING_CENTERED, # Default is MicrophonePosition.MONITOR
    do_step_1=True,
    do_step_2=False,
    do_step_3=False,
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepaths)

## Custom dialog directory names

In [ ]:
dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    room=room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
    # microphone_position=MicrophonePosition.MONITOR # Default is MicrophonePosition.MONITOR
    microphone_position=MicrophonePosition.CEILING_CENTERED, # Default is MicrophonePosition.MONITOR
    do_step_1=True,
    do_step_2=True,
    do_step_3=True,
    dialog_dir_name="demo_dialog"
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepaths)

## Custom room accoustic audio file name

You can customize the name of the file in order to fit with the setup of the room, microphone position and so on.

In [ ]:
# dialog: AudioDialog = AudioDialog.from_dialog(original_dialog)
dialog: AudioDialog = audio_pipeline.inference(
    dialog,
    room=room, # Need to provide a room object to trigger the 3rd step of the audio pipeline
    # microphone_position=MicrophonePosition.MONITOR # Default is MicrophonePosition.MONITOR
    microphone_position=MicrophonePosition.CEILING_CENTERED, # Default is MicrophonePosition.MONITOR
    do_step_1=False,
    do_step_2=False,
    do_step_3=True,
    dialog_dir_name="demo_dialog",
    room_name="my_room_acc_1"
)
print(dialog.audio_step_1_filepath)
print(dialog.audio_step_2_filepath)
print(dialog.audio_step_3_filepaths)